In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers

import os

import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224

data_dir = 'Spine/'

X = []
y = []
for folder in os.scandir(data_dir):
    print(folder)

    for entry in os.scandir(data_dir + folder.name):
        img = cv2.imread(data_dir + folder.name + '/' + entry.name, cv2.IMREAD_COLOR)  # Read image using OpenCV
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = np.array(img, dtype='float32')
        img = img / 255
        X.append(img)
        if folder.name == 'normal':
            y.append(0)
        elif folder.name == 'osteophenia':
            y.append(1)
        else:
            y.append(2)

X = np.array(X)
y = np.array(y)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

batch_size = 32
num_batches = 60  

augmented_images = []
augmented_labels = []
for _ in range(num_batches):
    X_batch, y_batch = next(datagen.flow(X, y, batch_size=batch_size, shuffle=False))
    augmented_images.append(X_batch)
    augmented_labels.append(y_batch)

X_aug= np.concatenate(augmented_images, axis=0)
y_aug = np.concatenate(augmented_labels, axis=0)
X_augu = np.concatenate((X, X_aug), axis=0)
y_augu=np.concatenate((y, y_aug), axis=0)



D:\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<DirEntry 'normal'>
<DirEntry 'osteophenia'>
<DirEntry 'osteoporosis'>


In [3]:

import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) 
x = Dense(128, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

opt = Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=["accuracy"])


In [4]:

X_train_val, X_test, y_train_val, y_test = train_test_split(X_augu, y_augu, test_size=0.2, shuffle=True, stratify=y_augu, random_state=123)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, shuffle=True, stratify=y_train_val, random_state=123)

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32, shuffle=True)


Epoch 1/10
42/42 [==============================] - 56s 1s/step - loss: 5.3296 - accuracy: 0.8881 - val_loss: 0.3008 - val_accuracy: 0.9463
Epoch 2/10
42/42 [==============================] - 50s 1s/step - loss: 0.3343 - accuracy: 0.9410 - val_loss: 0.2021 - val_accuracy: 0.9463
Epoch 3/10
42/42 [==============================] - 51s 1s/step - loss: 0.2409 - accuracy: 0.9440 - val_loss: 0.1797 - val_accuracy: 0.9463
Epoch 4/10
42/42 [==============================] - 51s 1s/step - loss: 0.2276 - accuracy: 0.9455 - val_loss: 0.1719 - val_accuracy: 0.9463
Epoch 5/10
42/42 [==============================] - 50s 1s/step - loss: 0.2184 - accuracy: 0.9433 - val_loss: 0.1712 - val_accuracy: 0.9463
Epoch 6/10
42/42 [==============================] - 49s 1s/step - loss: 0.2658 - accuracy: 0.9448 - val_loss: 0.2666 - val_accuracy: 0.9463
Epoch 7/10
42/42 [==============================] - 49s 1s/step - loss: 0.2926 - accuracy: 0.9448 - val_loss: 0.2477 - val_accuracy: 0.9463
Epoch 8/10
42/42 [==

In [6]:

test_loss, test_accuracy = model.evaluate(X_val,y_val)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



11/11 [==============================] - 10s 876ms/step - loss: 0.1761 - accuracy: 0.9463
Test Loss: 0.176095113158226
Test Accuracy: 0.9462686777114868


In [7]:

model.save('Spine_InceptionV3_with_aug.h5')



D:\anaconda\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
